In [1]:
# Loading modelvis.ms and gaintable applied ms

## Params used to generated model.ms
# {
#         "beam_type": None,
#         "eb_ms": None,
#         "eb_coeffs": None,
#         "gleamfile": None,
#         "lsm_csv_path": "/home/ska/Work/data/INST/lg3/sky_model_cal.csv",
#         "fov": 10.0,
#         "flux_limit": 1.0,
#         "alpha0":-0.78,
#         "reset_vis": False,
#         "export_model_vis": False,
#     }

input_ms = "/home/ska/Work/data/INST/lg3/cal_bpp_vis-lg3-rotated.small.ms"
model_ms = "/home/nitin/Work/ska/sdp/ska-sdp-instrumental-calibration/model.ms"
applied_ms = "/home/nitin/Work/ska/sdp/ska-sdp-instrumental-calibration/cal_bpp_vis-lg3-rotated.small.applied.ms"

In [3]:
import matplotlib.pyplot as plt
from ska_sdp_instrumental_calibration.data_managers.dask_wrappers import load_ms

In [4]:
uncalibrated_ms = load_ms(input_ms, fchunk=32)
model = load_ms(model_ms, fchunk=32)
applied = load_ms(applied_ms, fchunk=32)

In [5]:
def create_amp_freq_plots(ds, applied, title, subtitle1_prefix, subtitle2_prefix, time, baseline_start, baselines_count, filename_prefix):
    fig = plt.figure(layout="constrained",figsize=(20,10))
    fig.suptitle(title, fontsize=16)
    subfigs = fig.subfigures(1,baselines_count).reshape(-1)

    baselineid = baseline_start
    for idx,subfig in enumerate(subfigs):
        fig1,fig2 = subfig.subplots(2, 1)
        
        fig1.set_title(f"{subtitle1_prefix}. baselineid {baselineid}")
        fig1.set_xlabel("channel")
        fig1.set_ylabel("amp")
        fig1.plot(abs(ds.vis.isel(polarisation=0, baselineid=baselineid, time=time)))

        fig2.set_title(f"{subtitle2_prefix}. baselineid {baselineid}")
        fig2.set_xlabel("channel")
        fig2.set_ylabel("amp")
        fig2.plot(abs(applied.vis.isel(polarisation=0, baselineid=baselineid, time=time)))
        
        baselineid += 1
    
    fig.savefig(f"{filename_prefix}-baseline-{baseline_start}-{baseline_start+baselines_count}.png")
    plt.close()

## Amp vs Freq plots

In [6]:
# amp vs freq input vs applied flux cal

create_amp_freq_plots(uncalibrated_ms,
                      applied,
                      title="Amp vs Channel. uncalibrated vs calibrated data",
                      subtitle1_prefix="Uncalibrated",
                      subtitle2_prefix="Calibrated",
                      time=0,
                      baseline_start=2,
                      baselines_count=3,
                      filename_prefix="amp-freq-uncalibrated-vs-calibrated"
                     )

In [7]:
# amp vs freq model vs applied flux cal

create_amp_freq_plots(model,
                      applied,
                      title="Amp vs Channel. Model vs calibrated data",
                      subtitle1_prefix="Model",
                      subtitle2_prefix="After flux cal",
                      time=0,
                      baseline_start=2,
                      baselines_count=3,
                      filename_prefix="amp-freq-model-vs-calibrated"
                     )

## Amp vs UVWaves plots

In [8]:
# amp vs uv wave

def get_uv_wave(uvw, frequency):
    c = 3e8
    wavelength = c / frequency
    uvw_t = uvw.transpose("spatial", "time", "baselineid")
    return ((uvw_t[0] ** 2 + uvw_t[1] ** 2) ** 0.5) / wavelength
    

In [17]:
def plot_uv_waves(model, applied):
    fig = plt.figure(layout="constrained",figsize=(10,5))
    fig.suptitle("Amp vs UVWave", fontsize=16)
    model_fig,applied_fig = fig.subplots(1, 2)

    model_fig.set_title("Model")
    model_fig.set_xlabel("UVwave (λ)")
    model_fig.set_ylabel("amp")
    
    model_fig.scatter(
        abs(get_uv_wave(model.uvw, model.frequency).stack(flatted_dim=("time", "baselineid", "frequency"))),
        abs(model.vis.isel(polarisation=0).stack(flatted_dim=("time", "baselineid", "frequency"))),
        s=1.0
    )
    
    applied_fig.set_title("After flux cal")
    applied_fig.set_xlabel("UVwave (λ)")
    applied_fig.set_ylabel("amp")
    applied_fig.scatter(
        abs(get_uv_wave(applied.uvw, applied.frequency).stack(flatted_dim=("time", "baselineid", "frequency"))),
        abs(applied.vis.isel(polarisation=0).stack(flatted_dim=("time", "baselineid", "frequency"))),
        s=1.0
    )

    fig.savefig("uv-distance-model-flux-cal.png")
    plt.close()
    

In [18]:
plot_uv_waves(model, applied)